In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pydantic import BaseModel, Field
from typing import Dict, List
from langchain_community.tools import DuckDuckGoSearchResults

In [3]:
# 상태 정보 정의

class State(BaseModel):
    company_name: str = Field(description="회사 이름")
    ticker: str = Field(description="회사 티커", default="")
    segment: Dict[str, dict] = Field(description="사업부 정보", default_factory=dict)
    PER: float = Field(description="기업의 현재 PER(TTM)", default=0)
    peer_list: List[str] = Field(description="경쟁사 리스트", default_factory=list)
    average_peer_PER: float = Field(description="경쟁사 PER 평균", default=0)
    result: Dict[str, str] = Field(description="결과", default_factory=dict)


In [4]:
from yahooquery import search
from deep_translator import GoogleTranslator

# 한글 문자 범위를 이용해 한글 포함 여부 확인
def contains_korean(text):
    for char in text:
        if '\uac00' <= char <= '\ud7a3' or '\u3131' <= char <= '\u318e':
            return True
    return False

def get_ticker(company_name):
    try:            
        # 한글 포함 여부 확인
        is_korean = contains_korean(company_name)
        if is_korean:
            # 회사명을 영어로 번역
            translated = GoogleTranslator(source='auto', target='en').translate(company_name)
            
            # 번역된 이름으로 검색
            results = search(translated)
        else:
            results = search(company_name)
            
        # KSC, NYSE, NASDAQ, AMEX, JPX, HKG 순서로 찾기
        for quote in results['quotes']:
            if quote['exchange'] == 'KSC': # 한국
                return quote['symbol']
            elif quote['exchange'] == 'NYQ': # NYSE
                return quote['symbol']
            elif quote['exchange'] == 'NMS': # NASDAQ
                return quote['symbol']
            elif quote['exchange'] == 'JPX': # 일본
                return quote['symbol']
            elif quote['exchange'] == 'HKG': # 홍콩
                return quote['symbol']
            else:
                continue
        
        # KSC, NYSE, NASDAQ, AMEX에 없으면 None 반환
        return None
    except Exception as e:
        print(f"Error translating or searching for {company_name}: {e}")
        return None

In [5]:
# 상태 정보 초기화, 티커 추출
company_name = "네이버"

ticker = get_ticker(company_name)
ticker = ticker.split(".")[0]
print(ticker)

005930


In [6]:
import pandas as pd
consensus_df = pd.read_csv(f"./consensus_crawling/consensus_result.csv")
current_quarter_sales_consensus = consensus_df.loc[consensus_df['종목코드'] == ticker, '직전분기_매출액_컨센서스'].values[0] * (10 ** 8) #(10^8 =억원)

In [7]:
state = State(company_name=company_name, ticker=ticker)

income_stmt_cum = pd.read_excel(f"./data/quarterly_financial_data/{ticker}_quarterly_financial_data.xlsx",header=0)

In [8]:
# 사업부 추출
segments = []
for item in income_stmt_cum.iloc[:, 0]:
    if item == '영업수익':
        break
    segments.append(item)

# YoY 계산 ((현재 - 이전) / 이전) * 100
present_quarter = income_stmt_cum.iloc[:,1+4]
year_ago_quarter = income_stmt_cum.iloc[:,1]
yoy = ((present_quarter - year_ago_quarter) / year_ago_quarter) * 100

# 각 사업부별 YoY와 사업부 이름을 함께 보여주기
for segment, growth in zip(income_stmt_cum['계정'], yoy):
    if segment in segments:  # segments 리스트에 있는 사업부만 출력
        state.segment[segment] = {"yoy":growth}

# 영업수익 yoy 계산
for segment, growth in zip(income_stmt_cum['계정'], yoy):
    if segment == '영업수익':  # segments 리스트에 있는 사업부만 출력
        state.segment['영업수익'] = {"yoy":growth}

# 사업부별 매출액 추출
for segment, sales in zip(income_stmt_cum['계정'], present_quarter):
    if segment in segments:
        state.segment[segment].update({"sales":sales})

# 영업수익 추출
for segment, sales in zip(income_stmt_cum['계정'], present_quarter):
    if segment == '영업수익':  
        state.segment['영업수익'].update({"sales":sales})

print(state.segment)

{'DX': {'yoy': 3.000059029116687, 'sales': 134357500000000}, 'DS': {'yoy': 80.31495186193965, 'sales': 80965200000000}, 'SDC': {'yoy': -1.3328141566349845, 'sales': 21031700000000}, 'Harman': {'yoy': -1.1009069102932885, 'sales': 10348900000000}, '영업수익': {'yoy': 17.748413234716548, 'sales': 225082634000000}}


In [9]:
# 사업부별 뉴스 검색
duckduckgo_search_tool = DuckDuckGoSearchResults(max_results=1,backend="news")
for segment in segments:
    news_result = duckduckgo_search_tool.invoke(
        {
            "query": f"{state.company_name} {segment} 사업부 매출"
        }
    )
    state.segment[segment].update({"news_result":news_result})

print(state.segment)

{'DX': {'yoy': 3.000059029116687, 'sales': 134357500000000, 'news_result': 'snippet: [서울=뉴시스]이인준 기자 = 삼성전자가 반도체 사업부문 직원들에게만 파격적인 성과급과 격려금을 지급하기로 한 것에 대해 노조가 거세게 반발하고 있다. 삼성전자 최대 노조인 전국삼성전자노동조합(전삼노)는 20일 입장문을 통해 "격려금 지급을 받지 못한 DX부문 직원들은 실망감과 박탈감을 느끼고 있다"며 회사가 직원 간 갈등을 초래하고 있다는 입장을 밝혔다., title: 삼성전자, 반도체만 격려금…노조 "DX 박탈감, 즉각 시정", link: https://www.msn.com/ko-kr/기술/소비자-전자-기기/삼성전자-반도체만-격려금-노조-dx-박탈감-즉각-시정/ar-AA1wddnx, date: 2024-12-20T08:48:03+00:00, source: NEWSIS on MSN.com, snippet: [디지털데일리 옥송이 기자] 삼성전자가 인사 및 조직 개편을 마무리 지으면서 내년 업무 채비를 갖췄다. 모바일 및 가전 등 제품을 모두 아우르는 DX(디바이스 경험)부문은 한종희 대표이사를 ..., title: \'한종희\' 삼성전자 DX, 내년 키워드 \'안정\'…영업익 개선 숙제, link: https://www.ddaily.co.kr/page/view/2024120515115760549, date: 2024-12-05T16:26:00+00:00, source: 디지털데일리, snippet: DX(디바이스경험) 부문의 경우 사업부별로 차등 지급된다. TV·모바일·의료기기·전장사업부가 75%로 가장 높고, 생활가전사업부 37.5%, 네트워크 ..., title: 삼성전자 메모리사업부 성과급 200% \'역대 최대\', link: https://zdnet.co.kr/view/?no=20241220110722, date: 2024-12-20T11:30:00+00:00, source: zdnet, snippet

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class yoy_prediction(BaseModel):
    """yoy 예측값을 나타내는 모델"""
    business_segment: str = Field(description="사업부")
    yoy: float = Field(description="사업부 yoy 예측값")
    reason: str = Field(description="사업부 yoy 예측값의 근거")

parser = PydanticOutputParser(pydantic_object=yoy_prediction)


template = """
당신은 증권사 소속 애널리스트입니다.
다음 내용을 참고하여 {company_name}의 {business_segment} 사업부의 매출 혹은 비용이 yoy로 얼마나 변할지 예측하시오.

직전 분기 yoy : {yoy}

{company_name}의 {business_segment} 사업부의 매출 혹은 비용에 큰 영향을 미치는 뉴스 : {news}

응답 형식:
{format_instructions}
"""

for segment in segments:
    prompt = ChatPromptTemplate.from_template(template=template, partial_variables={"format_instructions": parser.get_format_instructions()})

    llm = ChatOpenAI(temperature=0, model="gpt-4o")

    chain = prompt | llm | parser

    result = chain.invoke({"company_name": state.company_name, "news": state.segment[segment]['news_result'], "business_segment": segment, "yoy": state.segment[segment]['yoy']})

    # 예측값 및 예측 이유업데이트
    state.segment[segment].update({"yoy_prediction":result.yoy})
    state.segment[segment].update({"yoy_prediction_reason":result.reason})

print(state.segment)

{'DX': {'yoy': 3.000059029116687, 'sales': 134357500000000, 'news_result': 'snippet: [서울=뉴시스]이인준 기자 = 삼성전자가 반도체 사업부문 직원들에게만 파격적인 성과급과 격려금을 지급하기로 한 것에 대해 노조가 거세게 반발하고 있다. 삼성전자 최대 노조인 전국삼성전자노동조합(전삼노)는 20일 입장문을 통해 "격려금 지급을 받지 못한 DX부문 직원들은 실망감과 박탈감을 느끼고 있다"며 회사가 직원 간 갈등을 초래하고 있다는 입장을 밝혔다., title: 삼성전자, 반도체만 격려금…노조 "DX 박탈감, 즉각 시정", link: https://www.msn.com/ko-kr/기술/소비자-전자-기기/삼성전자-반도체만-격려금-노조-dx-박탈감-즉각-시정/ar-AA1wddnx, date: 2024-12-20T08:48:03+00:00, source: NEWSIS on MSN.com, snippet: [디지털데일리 옥송이 기자] 삼성전자가 인사 및 조직 개편을 마무리 지으면서 내년 업무 채비를 갖췄다. 모바일 및 가전 등 제품을 모두 아우르는 DX(디바이스 경험)부문은 한종희 대표이사를 ..., title: \'한종희\' 삼성전자 DX, 내년 키워드 \'안정\'…영업익 개선 숙제, link: https://www.ddaily.co.kr/page/view/2024120515115760549, date: 2024-12-05T16:26:00+00:00, source: 디지털데일리, snippet: DX(디바이스경험) 부문의 경우 사업부별로 차등 지급된다. TV·모바일·의료기기·전장사업부가 75%로 가장 높고, 생활가전사업부 37.5%, 네트워크 ..., title: 삼성전자 메모리사업부 성과급 200% \'역대 최대\', link: https://zdnet.co.kr/view/?no=20241220110722, date: 2024-12-20T11:30:00+00:00, source: zdnet, snippet

In [12]:
# 다음 분기(예측) 열 추가 (모든 행 0으로 초기화)
income_stmt_cum['next_quarter'] = 0

# segments에 있는 계정에 대해서만 yoy_prediction을 적용하여 업데이트
for segment in segments:
    yoy_prediction = state.segment[segment]['yoy_prediction']
    predicted_growth_rate = 1 + (yoy_prediction / 100)  # yoy_prediction을 성장률로 변환
    
    # 해당 segment 행의 next_quarter 값을 업데이트(next_quarter의 1년 전 분기 값 * 성장률)
    income_stmt_cum.loc[income_stmt_cum['계정'] == segment, 'next_quarter'] = income_stmt_cum.loc[income_stmt_cum['계정'] == segment].iloc[:,2] * predicted_growth_rate

# next_quarter의 segments 행들의 합계 계산
revenue_sum = income_stmt_cum.loc[income_stmt_cum['계정'].isin(segments), 'next_quarter'].sum()

# segments의 합계를 영업수익 값으로 업데이트
income_stmt_cum.loc[income_stmt_cum['계정'] == '영업수익', 'next_quarter'] = revenue_sum

# 직전분기의 영업수익 대비 영업이익 비율 계산
revenue_current_quarter = income_stmt_cum.loc[income_stmt_cum['계정'] == '영업수익'].iloc[:,5].values[0]
operating_profit_current_quarter = income_stmt_cum.loc[income_stmt_cum['계정'] == '영업이익'].iloc[:,5].values[0]
operating_profit_margin = operating_profit_current_quarter / revenue_current_quarter

# 동기간 전분기의 영업이익율을 다음 분기의 영업이익 예측에 사용(영업이익률을 그대로 사용)
revenue_next_quarter = income_stmt_cum.loc[income_stmt_cum['계정'] == '영업수익', 'next_quarter'].values[0]
operating_profit_next_quarter = revenue_next_quarter * operating_profit_margin

# 영업이익 행의 next_quarter 값을 업데이트
income_stmt_cum.loc[income_stmt_cum['계정'] == '영업이익', 'next_quarter'] = operating_profit_next_quarter

# 직전분기의 순이익률 계산
net_income_current_quarter = income_stmt_cum.loc[income_stmt_cum['계정'] == '순이익'].iloc[:,5].values[0]
net_margin_current_quarter = net_income_current_quarter / revenue_current_quarter

# 다음분기의 순이익 계산(직전분기의 순이익률 * 다음분기의 추정 영업수익)
net_income_next_quarter = revenue_next_quarter * net_margin_current_quarter

# 순이익 행 업데이트
income_stmt_cum.loc[income_stmt_cum['계정'] == '순이익', 'next_quarter'] = net_income_next_quarter

income_stmt_cum

/var/folders/f7/84_7pkk50r10snsc2qr066600000gn/T/ipykernel_14592/669459975.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '39997466865236.95' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  income_stmt_cum.loc[income_stmt_cum['계정'] == '영업이익', 'next_quarter'] = operating_profit_next_quarter


,계정,2023-3Q,2023-4Q,2024-1Q,2024-2Q,2024-3Q,next_quarter
0,DX,130444100000000,169992300000000,47292700000000,89363600000000,134357500000000,1.747521e+14
1,DS,44902100000000,66594500000000,23137300000000,51693800000000,80965200000000,1.231998e+14
2,SDC,21315800000000,30975400000000,5386400000000,13032600000000,21031700000000,3.099089e+13
3,Harman,10464100000000,14388500000000,3200300000000,6819300000000,10348900000000,1.423742e+13
4,영업수익,191155556000000,258935494000000,71915601000000,145983903000000,225082634000000,3.431802e+14
5,영업이익,3742259000000,6566976000000,6606009000000,17049887000000,26233258000000,3.999747e+13
6,순이익,9142342000000,15487100000000,6754708000000,16596053000000,26696957000000,4.070446e+13


In [13]:
# income_stmt_cum 데이터프레임을 엑셀 파일로 저장
income_stmt_cum.to_excel('income_stmt_cum.xlsx', index=False)

In [14]:
#직전분기 매출액(누적) 컨센서스와 실제값 저장

def naver_sales_earning_surprise(current_quarter_sales_consensus:float) -> float:
    """네이버 증권 -> 종목분석 -> 컨센서스 -> 어닝서프라이즈(매출액) -> 분기
        return :
        직전 분기 매출액에 대한 발표직전 컨센서스 (단위: 억원) + 직전 분기의 전분기까지의 실제 매출액 = 직전 분기까지 누적 매출액 컨센서스
    """
    
    before_current_quarter_sales_cum = income_stmt_cum.loc[income_stmt_cum['계정'] == '영업수익'].iloc[:,4].values[0]
    current_quarter_sales_cum_consensus = current_quarter_sales_consensus + before_current_quarter_sales_cum

    return current_quarter_sales_cum_consensus

current_quarter_sales_cum_consensus = naver_sales_earning_surprise(current_quarter_sales_consensus)
current_quarter_sales_cum = state.segment['영업수익']['sales']

year_ago_quarter_sales_cum = income_stmt_cum.loc[income_stmt_cum['계정'] == '영업수익'].iloc[:,1].values[0]
yoy_consensus = ((current_quarter_sales_cum_consensus - year_ago_quarter_sales_cum) / year_ago_quarter_sales_cum) * 100

state.segment['영업수익'].update({"yoy_consensus":yoy_consensus})
state.segment['영업수익'].update({"sales_consensus":current_quarter_sales_cum_consensus})

state.segment['영업수익']

{'yoy': 17.748413234716548,
 'sales': 225082634000000,
 'yoy_consensus': 18.690849352032433,
 'sales_consensus': 226884153000000.0}

In [15]:
def current_quarter_review(state:State) -> None:
    """직전 분기 실적 리뷰를 작성"""

    result_of_business_segment = ""
    for segment in segments:
        result_of_business_segment += f"{segment} 사업부의 직전 분기 매출액은 {state.segment[segment]['sales']} 입니다. yoy로 {state.segment[segment]['yoy']}% 변했습니다.\n"

    result_of_total_business = f"총 매출액은 {state.segment['영업수익']['sales']} 입니다. yoy로 {state.segment['영업수익']['yoy']}% 변했습니다."

    consensus_of_total_business = f"총 매출액에 대한 컨센서스는 {state.segment['영업수익']['sales_consensus']} 였습니다. yoy로 {state.segment['영업수익']['yoy_consensus']}% 변화하는 것이 컨센서스였습니다."

    template = """
    당신은 증권사 소속 애널리스트입니다.
    다음 내용을 참고하여 직전 분기 실적에 대한 리뷰를 작성하시오.

    직전 분기 사업부별 매출 실적 : {result_of_business_segment}
    직전 분기 총 매출 실적 : {result_of_total_business}
    직전 분기 총 매출 실적에 대한 컨센서스 : {consensus_of_total_business}

    """
    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    chain = prompt | llm 
    review = chain.invoke({"result_of_business_segment":result_of_business_segment, "result_of_total_business":result_of_total_business, "consensus_of_total_business":consensus_of_total_business})
    state.result.update({"sales_review":review.content})

    return None

current_quarter_review(state)
print(state.result['sales_review'])

직전 분기 실적 리뷰

이번 분기 실적은 전반적으로 긍정적인 성과를 보였으나, 일부 사업부에서는 예상보다 저조한 실적을 기록하였습니다. 총 매출액은 225조 8263억 4천만 원으로, 전년 동기 대비 17.75% 증가하였습니다. 이는 시장 컨센서스인 226조 8841억 5천 3백만 원에 약간 미치지 못하는 결과입니다.

사업부별로 살펴보면, DX 사업부는 134조 3575억 원의 매출을 기록하며 전년 동기 대비 3.00% 증가하였습니다. 이는 안정적인 성장세를 유지하고 있음을 보여줍니다. 반면, DS 사업부는 80.31%라는 높은 성장률을 기록하며 80조 9652억 원의 매출을 달성하였습니다. 이는 반도체 수요 증가와 관련된 긍정적인 시장 환경이 반영된 결과로 보입니다.

SDC 사업부는 2조 1031억 7천만 원의 매출을 기록하며 전년 동기 대비 1.33% 감소하였습니다. 이는 디스플레이 시장의 경쟁 심화와 수요 감소가 영향을 미친 것으로 분석됩니다. Harman 사업부 역시 1조 3489억 원의 매출로 전년 동기 대비 1.10% 감소하였습니다. 이는 글로벌 자동차 시장의 불확실성과 관련된 요인들이 작용한 결과로 보입니다.

종합적으로, 이번 분기 실적은 DS 사업부의 강력한 성장에 힘입어 전반적인 매출 증가를 이끌었으나, SDC와 Harman 사업부의 부진이 다소 아쉬운 부분으로 남습니다. 향후 시장 변화에 대한 대응 전략이 필요할 것으로 보입니다.


In [16]:
# StructuredOutputParser(peer 리스트 출력)

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate

response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변, 파이썬 리스트 형식이어야 함."),
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [17]:
import yfinance as yf
import numpy as np

def find_peer(company: str) -> list[str]:
    prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
    )
    chain = prompt | llm | output_parser  # 프롬프트, 모델, 출력 파서를 연결
    peer_list = chain.invoke({"question": f"{company}와 사업구조가 비슷하고, 같은 산업 혹은 섹터에 속한 경쟁사는?"
                              "(코스피, 뉴욕거래소 등 상장된 회사만 찾으세요. 반드시 회사명만 출력해주세요.)"})
    return peer_list

def find_peer_PERs_tool(company: str) -> None:
    """기업과 동종 업계의 Peer Group PER 평균을 찾습니다."""
    ticker = get_ticker(company)
    peer_list = find_peer(company)['answer']
    if ticker is None:
        return None
    
    peer_pers = {}
    for peer in peer_list:
        ticker = get_ticker(peer)
        if ticker is None:
            continue
        elif ".KS" in ticker:
            ticker = yf.Ticker(ticker)
            earning_ttm = 0
            for i in range(4):
                earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
            trailingPERttm = ticker.info.get("marketCap")/earning_ttm
            if trailingPERttm <0 :
                continue
            peer_pers[peer] = trailingPERttm
        else:
            ticker = yf.Ticker(ticker)
            earning_ttm = 0
            try:
                for i in range(4):
                    earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
            except:
                print(f"{peer}의 Net Income Common Stockholders 문제 발생")
                continue
            trailingPERttm = ticker.info.get("marketCap")/earning_ttm*0.7 # 외국 주식의 경우 PER을 30% 할인
            if trailingPERttm <0 :
                continue
            if np.isnan(trailingPERttm):
                continue
            peer_pers[peer] = trailingPERttm

    valid_peer_pers = {}

    if state.PER < 0:
        valid_peer_pers = peer_pers
    else:
        for key, value in peer_pers.items():
            if value < 0.5 * state.PER or value > 2 * state.PER:
                continue
            else:
                valid_peer_pers[key] = value
                    
    print(valid_peer_pers)
    
    average_peer_per = sum(valid_peer_pers.values()) / len(valid_peer_pers)

    valid_peer_list = []
    for key, value in valid_peer_pers.items():
        valid_peer_list.append(key)

    state.peer_list = valid_peer_list
    state.average_peer_PER = average_peer_per

    return None

In [18]:
def find_PER_tool(company: str) -> None:
    """기업의 현재 PER(TTM)를 찾습니다."""
    ticker = get_ticker(company)
    if ticker is None:
        return None
    else:
        ticker = yf.Ticker(ticker)
        earning_ttm = 0
        for i in range(4):
            earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
        trailingPERttm = ticker.info["marketCap"]/earning_ttm
        state.PER = trailingPERttm
        return None


In [19]:
find_PER_tool(state.company_name)
find_peer_PERs_tool(state.company_name)

/var/folders/f7/84_7pkk50r10snsc2qr066600000gn/T/ipykernel_14592/4070218302.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
/var/folders/f7/84_7pkk50r10snsc2qr066600000gn/T/ipykernel_14592/139201608.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
/var/folders/f7/84_7pkk50r10snsc2qr066600000gn/T/ipykernel_14592/139201608.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will 

{'SK Hynix': 12.203223468138647, 'Micron Technology': 17.9369313994848, 'Qualcomm': 12.219905756813251}


In [20]:
state.peer_list
state.average_peer_PER

14.120020208145567

In [21]:
def segment_yoy_prediction_result(state:State) -> None:
    """사업부별 매출액 예측결과와 근거를 사업부별로 정리하여 작성"""
    segment_yoy_prediction_result = ""
    for segment in segments:
        segment_yoy_prediction_result += f"다음 분기 {segment} 사업부의 매출액은 yoy로 {state.segment[segment]['yoy_prediction']}% 변화할 것으로 예측됩니다.\n 직전 분기 {segment} 사업부의 매출액은 {state.segment[segment]['yoy']}% 변화했습니다.\n"
        segment_yoy_prediction_result += f"그 이유는 다음과 같습니다. {state.segment[segment]['yoy_prediction_reason']}\n"
        segment_yoy_prediction_result += f"이 예측은 다음과 같은 뉴스를 기반으로 이루어졌습니다. {state.segment[segment]['news_result']}\n"

    template = """
    당신은 증권사 소속 애널리스트입니다.
    다음 내용을 참고하여 각 사업부별 매출액 예측결과와 근거를 사업부별로 정리하여 작성하세요.
    근거를 서술할 때에는 근거가 된 뉴스를 반드시 언급하세요.
    다음 분기 매출액 yoy 증가율 예측결과와 직전분기 매출액 yoy 증가율을 비교하세요.

    사업부별 매출 예측 결과와 근거 : {segment_yoy_prediction_result}
    """
    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    chain = prompt | llm
    segment_yoy_prediction_result = chain.invoke({"segment_yoy_prediction_result": segment_yoy_prediction_result}).content
    state.result.update({"segment_yoy_prediction_result": segment_yoy_prediction_result})
    return None

segment_yoy_prediction_result(state)
print(state.result['segment_yoy_prediction_result'])

### DX 사업부
- **예측 매출액 YoY 변화율**: 2.8%
- **직전 분기 매출액 YoY 변화율**: 3.0001%
- **근거**: 최근 뉴스에 따르면, 삼성전자의 DX 사업부 내에서 반도체 사업부와 비교하여 불평등한 보너스 배분으로 인해 내부 불만이 발생하고 있습니다. 이는 직원들의 사기와 생산성에 부정적인 영향을 미칠 수 있으며, 이에 따라 예상 성장률이 약간 감소할 것으로 보입니다. 또한, 조직 안정성에 대한 집중과 영업 이익 개선의 도전 과제가 내년의 신중한 전망을 시사합니다.
- **참고 뉴스**: 
  - [삼성전자, 반도체만 격려금…노조 "DX 박탈감, 즉각 시정"](https://www.msn.com/ko-kr/기술/소비자-전자-기기/삼성전자-반도체만-격려금-노조-dx-박탈감-즉각-시정/ar-AA1wddnx), NEWSIS on MSN.com, 2024-12-20
  - ['한종희' 삼성전자 DX, 내년 키워드 '안정'…영업익 개선 숙제](https://www.ddaily.co.kr/page/view/2024120515115760549), 디지털데일리, 2024-12-05

### DS 사업부
- **예측 매출액 YoY 변화율**: 85.0%
- **직전 분기 매출액 YoY 변화율**: 80.315%
- **근거**: 삼성전자의 DS 사업부는 200%의 성과급과 격려금 지급을 통해 강력한 실적을 보여주고 있습니다. 이러한 조치는 사업부가 상당한 재정적 성공을 거두었음을 시사하며, 이는 YoY 성장률 증가로 이어질 가능성이 큽니다. 또한, 최근의 임원 인사와 조직 안정성도 DS 사업부의 긍정적인 전망을 뒷받침합니다.
- **참고 뉴스**: 
  - [삼성전자 DS, 격려금 200만원 지급…메모리 성과급 200%(종합)](https://www.fnnews.com/news/202412201141203457), 파이낸셜뉴스, 2024-12-20
  - [삼성전자, 메모리사업부에 성과급 200%…DS부문 역대 최대](https://biz.ch

In [22]:
def valuation_result(state:State) -> None:
    """목표 주가 컨센서스 작성"""
    ticker = get_ticker(state.company_name)
    ticker = yf.Ticker(ticker)
    estEarnings = income_stmt_cum.loc[income_stmt_cum['계정'] == '순이익','next_quarter'].values[0]
    shares = ticker.info.get("sharesOutstanding")
    estEPS = estEarnings/shares
    price_consensus = ticker.analyst_price_targets

    template = """
    당신은 증권사 소속 애널리스트입니다.
    다음 내용을 참고하여 {company_name}의 목표 주가를 설정하고 그 이유를 반드시 서술하세요.
    직전분기 매출 실적 리뷰 내용과 사업부별 매출 예측 결과와 근거는 반드시 서술하세요.
    목표 주가를 설정할 때에는 먼저, 현재 기업 PER과 경쟁사들의 평균 PER을 반드시 고려하여 기업의 목표 PER을 설정하세요.
    목표 PER과 추정 EPS를 곱하여 목표 주가를 설정하세요.

    직전 분기 매출 실적 리뷰: {current_quarter_review_result}

    사업부별 매출 예측 결과와 근거 : {segment_yoy_prediction_result}

    현재 기업 PER : {PER}

    기업 경쟁사 리스트 : {peer_list}

    현재 경쟁사들의 평균 PER : {average_peer_PER}

    추정 EPS : {EPS} (추정 매출액을 기반으로 계산된 결과입니다.)

    목표 주가를 제안한 이후, 목표 주가 컨센서스를 아래 내용을 참고하여 작성하세요.

    평균 목표 주가 : {avg_price_consensus}
    최소 목표 주가 : {low_price_consensus}
    최대 목표 주가 : {high_price_consensus}
    현재 주가 : {current_price}
    """


    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    chain = prompt | llm
    valuation_result = chain.invoke({"company_name": state.company_name, 
                "current_quarter_review_result": state.result['sales_review'], 
                "segment_yoy_prediction_result": state.result['segment_yoy_prediction_result'], 
                "PER": state.PER, 
                "average_peer_PER": state.average_peer_PER,
                "peer_list": state.peer_list,
                "EPS": estEPS,
                "avg_price_consensus": price_consensus['mean'],
                "low_price_consensus": price_consensus['low'],
                "high_price_consensus": price_consensus['high'],
                "current_price": ticker.info.get("regularMarketPrice")})
    
    state.result.update({"valuation_result": valuation_result.content})
    return None

valuation_result(state)
print(state.result['valuation_result'])

삼성전자의 목표 주가를 설정하기 위해, 먼저 사업부별 매출 실적 리뷰와 예측 결과를 분석하겠습니다.

### 직전 분기 매출 실적 리뷰
- **총 매출액**: 225조 8263억 4천만 원으로 전년 동기 대비 17.75% 증가하였으나, 시장 컨센서스에는 약간 미치지 못했습니다.
- **DX 사업부**: 134조 3575억 원의 매출로 전년 동기 대비 3.00% 증가, 안정적인 성장세 유지.
- **DS 사업부**: 80조 9652억 원의 매출로 80.31%의 높은 성장률 기록, 반도체 수요 증가에 기인.
- **SDC 사업부**: 2조 1031억 7천만 원의 매출로 전년 동기 대비 1.33% 감소, 디스플레이 시장의 경쟁 심화와 수요 감소 영향.
- **Harman 사업부**: 1조 3489억 원의 매출로 전년 동기 대비 1.10% 감소, 글로벌 자동차 시장의 불확실성 영향.

### 사업부별 매출 예측 결과
- **DX 사업부**: YoY 변화율 2.8%로 약간 감소 예상, 내부 불만과 조직 안정성 이슈가 성장률에 영향을 미칠 것으로 보임.
- **DS 사업부**: YoY 변화율 85.0%로 강력한 성장 지속 예상, 성과급 지급과 조직 안정성이 긍정적 요인.
- **SDC 사업부**: YoY 변화율 0.05%로 소폭 증가 예상, 전년 대비 개선 가능성 있음.
- **Harman 사업부**: YoY 변화율 -1.05%로 약간의 개선 예상되나 여전히 부정적.

### 목표 주가 설정
현재 삼성전자의 PER은 11.11이며, 경쟁사들의 평균 PER은 14.12입니다. 삼성전자의 DS 사업부의 강력한 성장과 안정적인 DX 사업부의 성과를 고려할 때, 목표 PER을 경쟁사 평균에 근접한 13.5로 설정하는 것이 적절하다고 판단됩니다.

- **목표 PER**: 13.5
- **추정 EPS**: 6818.42

따라서, 목표 주가는 다음과 같이 계산됩니다:
\[ \text{목표 주가} = \text{목표 PER} \times \text{추정 EPS} = 13.5 \ti

In [23]:
from datetime import datetime

title = f"{state.company_name} 리포트"
report = ""
report += f"{title}\n"
report += f"="*len(title)*2
report += f"\n\n"
for key, value in state.result.items():
    report += f"{value}\n\n"
    report += f"-"*50
    report += f"\n\n"

report += f"\n"
report += "본 자료는 사업보고서, 기사 등 신뢰할 수 있는 자료를 바탕으로 작성되었습니다.\n"
report += "그러나, AI의 의견이 반영되었으며 정확성이나 완전성을 보장할 수 없습니다.\n"
report += "본 자료는 참고용으로 투자자 자신의 판단과 책임하에 투자하시기 바랍니다.\n"
report += f"최종 리포트 작성일 : {datetime.now().strftime('%Y-%m-%d')}"

print(report)

삼성전자 리포트

직전 분기 실적 리뷰

이번 분기 실적은 전반적으로 긍정적인 성과를 보였으나, 일부 사업부에서는 예상보다 저조한 실적을 기록하였습니다. 총 매출액은 225조 8263억 4천만 원으로, 전년 동기 대비 17.75% 증가하였습니다. 이는 시장 컨센서스인 226조 8841억 5천 3백만 원에 약간 미치지 못하는 결과입니다.

사업부별로 살펴보면, DX 사업부는 134조 3575억 원의 매출을 기록하며 전년 동기 대비 3.00% 증가하였습니다. 이는 안정적인 성장세를 유지하고 있음을 보여줍니다. 반면, DS 사업부는 80.31%라는 높은 성장률을 기록하며 80조 9652억 원의 매출을 달성하였습니다. 이는 반도체 수요 증가와 관련된 긍정적인 시장 환경이 반영된 결과로 보입니다.

SDC 사업부는 2조 1031억 7천만 원의 매출을 기록하며 전년 동기 대비 1.33% 감소하였습니다. 이는 디스플레이 시장의 경쟁 심화와 수요 감소가 영향을 미친 것으로 분석됩니다. Harman 사업부 역시 1조 3489억 원의 매출로 전년 동기 대비 1.10% 감소하였습니다. 이는 글로벌 자동차 시장의 불확실성과 관련된 요인들이 작용한 결과로 보입니다.

종합적으로, 이번 분기 실적은 DS 사업부의 강력한 성장에 힘입어 전반적인 매출 증가를 이끌었으나, SDC와 Harman 사업부의 부진이 다소 아쉬운 부분으로 남습니다. 향후 시장 변화에 대한 대응 전략이 필요할 것으로 보입니다.

--------------------------------------------------

### DX 사업부
- **예측 매출액 YoY 변화율**: 2.8%
- **직전 분기 매출액 YoY 변화율**: 3.0001%
- **근거**: 최근 뉴스에 따르면, 삼성전자의 DX 사업부 내에서 반도체 사업부와 비교하여 불평등한 보너스 배분으로 인해 내부 불만이 발생하고 있습니다. 이는 직원들의 사기와 생산성에 부정적인 영향을 미칠 수 있으며, 이에 따라 예상 성장률이 약간 감소할 것으로 보입니다.